In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from cyvlfeat.sift import dsift
from cyvlfeat.kmeans import kmeans

In [ ]:
#https://github.com/menpo/cyvlfeat/blob/master/cyvlfeat/sift/dsift.py

In [3]:
train_path = 'hw5_data/train/'
test_path = 'hw5_data/test/'
size = 32

#load train data
feature = []
label = []
allList = os.listdir(train_path)
num = 0
for item in allList:
    if os.path.isdir(train_path + item):
        data_of_class = os.listdir(train_path + item)
        f = []
        i = 0
        for data in data_of_class:
            img = cv2.imread(train_path + item+'/' + data,0)
            gray = img
            #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            #gray = cv2.resize(gray,(size,size),interpolation=cv2.INTER_AREA)
            #normalize
            #gray = gray - np.mean(gray)
            #gray = gray/np.std(gray)
            f.append(gray)
            i+=1
            num+=1
        feature.append(f)
        label.append(item)

In [4]:
#load test data

test_feature = []
test_label = []
test_List = os.listdir(test_path)
for item in test_List:
    if os.path.isdir(test_path + item):
        data_of_class = os.listdir(test_path + item)
        f=[]
        i = 0
        for data in data_of_class:
            img = cv2.imread(test_path + item+'/' + data,0)
            gray=img
            #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            #gray = cv2.resize(gray,(size,size),interpolation=cv2.INTER_AREA)
            #normalize
            #gray = gray - np.mean(gray)
            #gray = gray/np.std(gray)
            f.append(gray)
            i+=1
        test_feature.append(f)
        test_label.append(item)

In [7]:
#train data
descriptors = []
#sift = cv2.xfeatures2d.SIFT_create(nfeatures = 200)
for i in range(len(feature)):
    d = []
    for j in range(len(feature[i])):
        t_kp,t_des = dsift(feature[i][j],step=15,size=8,fast=True,float_descriptors=True)
        #t_kp, t_des = sift.detectAndCompute(feature[i][j],None)
        d.append(t_des)
    descriptors.append(d)

In [8]:
# Stack all the descriptors vertically in a numpy array
des_np = descriptors[0][0]
for i in range(len(descriptors)):
    for j in range(len(descriptors[i])):
        dd = descriptors[i][j]
        if i!=0 and j!=0:
            des_np = np.vstack((des_np,dd))

In [9]:
# Perform k-means clustering
#criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,10,1.0)
#flags = cv2.KMEANS_RANDOM_CENTERS
#compactness,labels,centers = cv2.kmeans(des_np,400,None,criteria,10,flags)
centers = kmeans(des_np,400)

In [10]:
# Calculate the histogram of features
im_features = np.zeros((len(descriptors)*100,400),float)
for i in range(len(descriptors)):
    for j in range(len(descriptors[i])):
        for k in range(len(descriptors[i][j])):
            dist = np.sqrt(np.sum((centers - descriptors[i][j][k])**2,axis=1))
            closet = np.argmin(dist,axis=0)
            im_features[i*100+j,closet] += 1

In [85]:
#normalize histogram
#im_features = im_features / np.sum(im_features,axis = 1).reshape(-1,1)

In [38]:
#test data descriptors
test_descriptors = []
#sift = cv2.xfeatures2d.SIFT_create(nfeatures = 100)
for i in range(len(test_feature)):
    d = []
    for j in range(len(test_feature[i])):
        t_kp,t_des = dsift(test_feature[i][j],step=10,size=8,fast=True,float_descriptors=True)
        #t_kp, t_des = sift.detectAndCompute(test_feature[i][j],None)
        d.append(t_des)
    test_descriptors.append(d)

In [39]:
# Stack all the descriptors vertically in a numpy array
test_des_np = test_descriptors[0][0]
for i in range(len(test_descriptors)):
    for j in range(len(test_descriptors[i])):
        dd = test_descriptors[i][j]
        if i!=0 and j!=0:
            test_des_np = np.vstack((test_des_np,dd))

In [40]:
# Calculate the histogram of features
test_im_features = np.zeros((len(test_descriptors)*10,400),float)
for i in range(len(test_descriptors)):
    for j in range(len(test_descriptors[i])):
        for k in range(len(test_descriptors[i][j])):
            dist = np.sqrt(np.sum((centers - test_descriptors[i][j][k])**2,axis=1))
            closet = np.argmin(dist,axis=0)
            test_im_features[i*10+j,closet] += 1

In [86]:
#normalize histogram
#test_im_features = test_im_features / np.sum(test_im_features,axis = 1).reshape(-1,1)

In [41]:
from svmutil import *

In [42]:
gt = np.zeros((1500,1),int)
for i in range(len(feature)):
    for j in range(len(feature[i])):
        gt[i*100+j,0] = i

In [43]:
#training model
model=svm_train(gt.reshape(-1),im_features,'-q '+'-t 0') #-q: 算法default

In [44]:
gt_test = np.zeros((150,1),int)
for i in range(len(test_feature)):
    for j in range(len(test_feature[i])):
        gt_test[i*10+j,0] = i

In [45]:
p_label,p_acc,p_vals=svm_predict(gt_test.reshape(-1),test_im_features,model,'-q')

In [46]:
print('Accuracy: {}%'.format(p_acc[0]))

Accuracy: 63.33333333333333%
